<a href="https://colab.research.google.com/github/trancenoid/GANs/blob/master/Copytam_Da_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import io
text = io.open('lyrics.txt', ).read().lower()
Tx = 40
strides = 3

def prep_XY(text, Tx = 40, stride = 5):
  X = []
  Y = []
  for i in range(0,len(text) - Tx, stride):
      X.append(text[i : i + Tx])
      Y.append(text[i + Tx])
  return X,Y

X,Y = prep_XY(text, Tx, strides)

In [2]:
print(X[1], Y[1])

ool jigar
ya baba 
qaatil hai nazar
ya b a


In [19]:
chars = sorted(list(set(text)))
idx_char = dict(enumerate(chars))
char_idx = dict((c,i) for i,c in enumerate(chars))
print(char_idx)

{'\n': 0, ' ': 1, '!': 2, "'": 3, ',': 4, '-': 5, '.': 6, '/': 7, ':': 8, '?': 9, '[': 10, ']': 11, 'a': 12, 'b': 13, 'c': 14, 'd': 15, 'e': 16, 'f': 17, 'g': 18, 'h': 19, 'i': 20, 'j': 21, 'k': 22, 'l': 23, 'm': 24, 'n': 25, 'o': 26, 'p': 27, 'q': 28, 'r': 29, 's': 30, 't': 31, 'u': 32, 'v': 33, 'w': 34, 'y': 35, 'z': 36}


In [20]:
import numpy as np

def vectorize(X,Y, char_idx, Tx = 40):
  x = np.zeros((len(X), Tx, len(char_idx)))
  y = np.zeros((len(X), len(char_idx)))
  for i,sx in enumerate(X):
    for j,c in enumerate(sx):
      x[i, j, char_idx[c]] = 1
    y[i, char_idx[Y[i]]] = 1
  return x,y

x,y = vectorize(X,Y, char_idx, 40)
print(x.shape)

def sample(preds, temp = 1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds)/temp
  exp_pred = np.exp(preds)
  exp_pred = exp_pred/np.sum(exp_pred)
  choice = np.random.multinomial(1, exp_pred, 1)
  return np.argmax(choice)

(13684, 40, 37)


In [0]:
from keras.layers import LSTM, Dense
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.callbacks import LambdaCallback
opt = RMSprop(lr = 0.01)
model = Sequential()
model.add(LSTM(128, input_shape = (Tx, len(char_idx))))
model.add(Dense(len(chars), activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = opt)

In [0]:
from tqdm import tqdm
def on_epoch_end(epoch, _ ):
  if(epoch%10 != 0): return
  seed = "tu nazm nazm sa mere honthon pe theharja"
  seed = seed.zfill(40)
  
  cur_seq = seed
  diversity = [1.0]
  
  for temp in diversity:
    generated = ''
    generated += seed + " "
    
    for _ in tqdm(range(400)):
      x_pred = np.zeros((1, Tx, len(char_idx)))
      for i, ch in enumerate(cur_seq):
        if(ch != '0'):
          x_pred[0, i, char_idx[ch]] = 1
      pred = model.predict(x_pred)[0]
      chosen_idx = sample(pred, temp)
      pred_ch = idx_char[chosen_idx]

      cur_seq = cur_seq[1:] + pred_ch
      generated += pred_ch

    print(generated)
    
print_callback = LambdaCallback(on_epoch_end = on_epoch_end)
model.fit(x,y, epochs = 200, batch_size = 128, callbacks = [print_callback])
    
  

Epoch 1/200
13684/13684 [==============================] - 10s 754us/step - loss: 2.6277


100%|██████████| 400/400 [00:12<00:00, 33.30it/s]


tu nazm nazm sa mere honthon pe theharja areyi chiye paalz gaina
aipturi ya

uelarcsara
jaadi eeinn, hu mo hean naan kur sale
paradjaa
ai tere jhaiza
hooi chal balai
jo ja dheri
kua
po raghgse,beki 
aan oo ha
oare ter.neda meer 
tulta sonsaar

zari dehzn rarsahaar
metzaaro hai sher tardil badbaaneberi pain ahilkra onhar, phiy
iaha
mar?hjartnr paribhon

tohn oin meral jul k mekhe shsejny
te naraba
mu..ngte:n kaw ka
yuan

shujsa turbtoji h
Epoch 2/200
13684/13684 [==============================] - 9s 692us/step - loss: 2.1403
Epoch 3/200
13684/13684 [==============================] - 9s 691us/step - loss: 1.9188
Epoch 4/200
13684/13684 [==============================] - 9s 692us/step - loss: 1.7154
Epoch 5/200
13684/13684 [==============================] - 9s 688us/step - loss: 1.5260
Epoch 6/200
13684/13684 [==============================] - 9s 692us/step - loss: 1.3571
Epoch 7/200
13684/13684 [==============================] - 9s 692us/step - loss: 1.2073
Epoch 8/200
13684/13684 [=====

100%|██████████| 400/400 [00:12<00:00, 33.45it/s]


tu nazm nazm sa mere honthon pe theharja an main.
teri nigaaha ki dekh
badalon mein
tumse tu mera karta hain
tu itra mere
lagh jaaye
n manave k atana
dil ki diya

bhakoo sang deolu ki jina ki bhaba
bak, hawlon baaje
bhaine ki naaat hua tera
usiydig loon
khusm hai
toh babai
woon ki saja
paa kibbaan
nuki giye..


misa tere  koch bhi teri
jhangar karlan

judko tuda sun.. mein bin tere  
bin tere  
bin tere  
bin tere 

bandat re sunoon
sujo
Epoch 12/200
13684/13684 [==============================] - 9s 693us/step - loss: 0.7405
Epoch 13/200
 9984/13684 [====================>.........] - ETA: 2s - loss: 0.6700